# CSC324 Lecture 4

Recall how lazy evaluation works.  
We will use *Newton's method* to solve the cube root of a number.  
Consider the function: $$f(x) = x^3 - b$$ 
We try to approximate for x in $f(x) = 0$.  
$f^\prime(x) = 3x^2$  
So we can approximate $x1 = x - \frac{f(x)}{f^\prime(x)} = \frac{2x + \frac{b}{x^2}}{3}$  
The function below computes $x1$ from $x$ and it will return the cube root of b

In [1]:
cubeRoot b = within 0.001 (iterate next b)
  where
    next x = (2*x + b/x^2) / 3
    within eps (x : (x1 : rest))
        -- Some parethenses optional.
        -- Maybe show off @-pattern here.(f z
        | abs (x - x1) <= eps = x1
        | otherwise = within eps (x1 : rest)

In [2]:
cubeRoot 8

2.000000001071189

In [3]:
cubeRoot 2

1.2599210498953948

In [4]:
2 ** (1/3)

1.2599210498948732

As you can see, they are fairly close to each other.  
But more importantly, notice that even though the list is infinite, this function terminates!

`within 0.001` is considered to be the error margin.  
`within eps (x : (x1 : rest))`: We don't need to consider the case for an empty list as we know that the list is infinite

Python also has [lazy lists](https://stackoverflow.com/questions/2399194/python-lazy-list).

Notice `within 0.001 (iterate next b)`. This is similar to composition of two functions in math.  
Haskell has compositions and it can be written as `within 0.001 . iterate next`.  
Generally, `(f . g) x = f (g x)`  

## Linked Lists
Single Linked list is garbage and for kids. It consumes up a lot of space and is very slow (in terms of accessing and inserting). Arrays are much better as we know.

Consider the linked list we defined earlier:  
`data MyList a = Nil | Cons a (MyList a)`
Every instance of MyList stores either Nil (x bits) or sizeof(Integer) + sizeof (MyList) so that is a lot.

However, in Haskell linked lists are used for several reasons. Because Haskell is lazy, linked lists are great for **control** structure. If things are carefully done, storing linked lists could only take up $\mathcal{O}(1)$ space!  
If things are done properly, then you can fit the entire linked list in a single register!  
The idea is that since Haskell is lazy, it only looks at "some" of the list and ignores the entire other list. And then once it's done, it repeats... and so on...

This is why you can have infinite lists in Haskell.  
`func [1 ... ] = look at 1 and ingore everything else... look at 2 and ignore everything else... and so on ... `

### Different type of list functions
##### Sources
Generating new lists. Examples include `repeat, cycle, replicate, iterate, unfoldr,  the [x..], [x..y] notation`
##### Transducers
Functions that take in lists and return new lists. Examples include `map, filter, scanl, scanr, (foldr too, sometimes) take, drop, splitAt, takeWhile, dropWhile, span, break, partition, zip, zipWith, unzip`
##### Sinks
Functions that are functions... Idk. Examples: `foldr, foldl, foldl', sum, product, maximum, minimum`

### Problems with Lazy Evaluation
However, lazy evaluations isn't utopia unfortunately :'(

Recall the function `mySumV2`

In [5]:
mySumV2 xs = g 0 xs
  where
    g accum [] = accum
    {- Induction step: The list is x:xs
       Induction hypothesis: for all a: g a xs = a + sum of xs

       How to compute accum + sum of (x:xs)?

       accum + sum of (x:xs)
     = accum + x + sum of xs
     = (accum + x) + sum of xs
     = g (accum + x) xs               by IH
    -}
    g accum (x:xs) = g (accum + x) xs

Line 1: Eta reduce
Found:
mySumV2 xs = g 0 xs
Why not:
mySumV2 = g 0Line 3: Use foldl
Found:
g accum [] = accum
g accum (x : xs) = g (accum + x) xs
Why not:
g accum xs = foldl (+) accum xs

For `mySumV2 [1,2,3]`  
This will be:  
```
mySumV2 (1 : 2 : 3 : [])
→ g 0 (1 : 2 : 3 : [])
→ g (0 + 1) (2 : 3 : []) # Space is accumaleted
→ g ((0 + 1) + 2) (3 : []) 
→ g (((0 + 1) + 2) + 3) []  # Big junk instead of just storing the sum
→ ((0 + 1) + 2) + 3 # Python/C would just be 5 instead of (0 + 1+ ...)
→ (1 + 2) + 3
→ 3 + 3
→ 6
```
This takes $\Omega(n)$ space

We can even show that `mySumV1` consumes linear space

However, this can be avoided, by using the `seq x y` function (sequencing) which takes in two parameters and evaluates to "weak head normal form" where "weak head normal form" means the following:  
Evaluate:  
* for built-in number types: until you have the number
* for algebraic data types: until you have a data constructor
* for functions: until you have a lambda  


The "head" refers to the data constructors /number

For the third case, here is an example:  
`seq (if 1 > 0 then (\x -> x + 1) else (\x -> x -1)`
Rather than evaluate both the lambdas, only after the condition is checked, the lambda is evaluated. So space is being saved

So we can rewrite the sum function in the following way

In [6]:
mySumV3 xs = g 0 xs
  where
    g accum [] = accum
    g accum (x:xs) = seq accum (g (accum + x) xs)

Line 1: Eta reduce
Found:
mySumV3 xs = g 0 xs
Why not:
mySumV3 = g 0

We can make this even better by using  
```
g accum (x:xs) = let a1 = accum + x
                    in seq a1 (g a1 xs)
```
In this case, we will have memory stored only for `a1`  
However, this is not good
```
g accum (x:xs) = seq (accum + x) (g (accum + x) xs)
```
In this case, the compiler does not notice (g **(accum + x)** xs) as one, so there will be memory for `accum` and `x`

`Data.List` also has `foldl'` which basically tries to save up memory.  
```
foldl' f z [] = z
foldl' f z (x:xs) = seq z (foldl' f (f z x) xs)
```

## Type classes
Haskell lets you overload equality checking for the data structure that you have defined.  

For example, `== and /=`:  
```
class Eq a where
    (==), (/=) :: a -> a -> Bool
```



For example, let's say we want to support equality for `Bool` type:  

```
instance Eq Bool where
    False == False   = True
    True == True     = True
    _ == _           = False 
```

In [7]:
data Boolean = Truee | Falsee
instance Eq Boolean where
    Falsee == Truee   = True
    Truee == Falsee     = True 
    _ == _           = False 

In [8]:
Truee == Falsee

True

In [9]:
Truee == Truee

False

**Note**: A class is not a type. In other words, a function can never be of type `f :: <Class> -> <Class> -> b`  
Similarly, a type is not a "subclass" either. The only relationship is membership.  


However, we can do this:

In [10]:
eq3 x y z = x==y && y == z
eq3 :: Eq a => a -> a -> a -> Bool

What it means is that the function **uses** `<Class>`. Therefore, my arguments better support `<Class>` otherwise the function will not work.  
In the above example, `Eq a` means `a` must support `Eq` otherwise the function will not work.

We defined `data MyList` previously. We will now make it support `Eq`

In [11]:
data MyIntegerList = INil | ICons Integer MyIntegerList

instance Eq MyIntegerList where
    INil == INil              = True
    ICons x xs == ICons y ys  = x == y && xs == ys
    _ == _                    = False
    

In [12]:
data MyList a = Nil | Cons a (MyList a)

instance Eq a => Eq (MyList a) where
    Nil == Nil              = True
    Cons x xs == Cons y ys  = x == y && xs == ys
                            -- "x == y" is when we need to assume Eq a.
    _ == _                    = False

Notice for `MyList`, we have to make sure that `a` has `Eq` implemented. Therefore, we have to include `Eq a => ...`  
In the case of `MyIntegerList`, we know that `Integer` has `Eq` implemented, thus we don't need to include `Eq a => ...`

There are also other types of classes like `Ord`, `Bounded`, `Enum` and `Show`.  
You can read more about it [here](https://www.haskell.org/onlinereport/haskell2010/haskellch6.html#x13-1270006.3)  
![alt text](https://www.haskell.org/onlinereport/haskell2010/haskell2x.png)

We can make our life easier by letting our compiler defining the classes for you.  

`data MyType = ... deriving (Eq, Ord, Bounded, Enum, Show, Read)`  
However, for other classes, it may not be trivial for the compiler.  
More about it can be read [here](https://www.haskell.org/onlinereport/haskell2010/haskellch11.html#x18-18200011)

## Number Types
Haskell has Int, Integer, Rational, Float (single-precision floating point),
Double (double-precision floating point), Complex a.

#### `Num`
* **Some Methods** : +, -, *
* **Instances** : All number types
#### `Integrals`
* **Some Methods** : div, mod
* **Instances**: Int, Integer
#### `Fractional`
* **Some Methods** : /
* **Instances** : Rational, Float, Double, Complex a  


You can learn more about Number Types [here](https://www.haskell.org/onlinereport/haskell2010/haskellch6.html#x13-1350006.4)

Consider the function:  
```
let xs :: [Double]
      xs = [1, 2, 3]
  in sum xs / length xs
```
What's wrong with this ?

In [18]:
let xs :: [Double] 
    xs = [1, 2, 3] in sum xs / length xs

<interactive>:2:32: error:
    • Couldn't match expected type ‘Double’ with actual type ‘Int’
    • In the second argument of ‘(/)’, namely ‘length xs’
      In the expression: sum xs / length xs
      In the expression:
        let
          xs :: [Double]
          xs = [...]
        in sum xs / length xs

`xs` is of type `Int` and `sum` only takes in `Int`

In [20]:
let xs :: [Double] 
    xs = [1, 2, 3] in sum xs / fromIntegral (length xs)

2.0

## Machine Epsilon
The following [code](https://gist.github.com/AndrewBarfield/2557034) is forced to write two copies of the same code for two number number types because C# is a stupid language.  

```
static void CalculateMachineEpsilonForFloat()
		{

			Console.WriteLine( "Float:" );

			float machineEpsilon = 1.0f;
			float x = 0.0f;
			int loopCount = 0;

			do
			{
				machineEpsilon /= 2.0f;
				x = 1.0f + machineEpsilon;
				loopCount++;
				Console.WriteLine( "\t" + loopCount.ToString( "00" ) + ") " + machineEpsilon.ToString() );
			}
			while ( x > 1.0 );

			Console.WriteLine( "\n\tMantissa Bit Count: " + loopCount );
			Console.WriteLine( "\tMachine epsilon for float: " + 2 * machineEpsilon );

		}

	
		static void CalculateMachineEpsilonForDouble()
		{

			Console.WriteLine( "Double:" );
			
			double machineEpsilon = 1.0;
			double x = 0.0;
			int loopCount = 0;

			do
			{
				machineEpsilon /= 2.0;
				x = 1.0 + machineEpsilon;
				loopCount++;
				Console.WriteLine( "\t" + loopCount.ToString( "00" ) + ") " + machineEpsilon.ToString() );

			}
			while ( x > 1.0 );

			Console.WriteLine( "\n\tMantissa Bit Count: " + loopCount );
			Console.WriteLine( "\tMachine epsilon for double: " + 2 * machineEpsilon );

		}
```

In [21]:
epsilon :: (Ord a, Fractional a) => a
epsilon = last notTooSmall
  where
    halves = iterate (/ 2) 1
    notTooSmall = takeWhile (\e -> 1 + e > 1) halves

epsilonDouble :: Double
epsilonDouble = epsilon

epsilonFloat :: Float
epsilonFloat = epsilon

Notice that you do not need to write the code twice like in C#